In [1]:
import glob
import netCDF4 as nc
import pandas as pd
import numpy as np
import xarray as xr
import xesmf as xe
import cartopy.crs as ccrs
import os

In [2]:
lat_bnds, lon_bnds = [25, 50], [-125, -70]
lat_lst = np.arange(lat_bnds[0], lat_bnds[1]+0.01, 0.5)
lon_lst = np.arange(lon_bnds[0], lon_bnds[1]+0.01, 0.5)
print(lat_lst[1]-lat_lst[0], lon_lst[1]-lon_lst[0])
print(len(lat_lst), len(lon_lst))
projection_dimension = [129, 135]

0.5 0.5
51 111


# CRU .nc File

In [7]:
files_list = glob.glob(rf"/home/uditbhatia/Documents/Sarth/CRU_tmp/cru_ts3.23.*.tmp.dat.nc")
# files_list
ds = xr.open_mfdataset(files_list,combine = 'by_coords')
print(ds.dims)
print(ds.coords)
print()
ds = ds.sel(time=((ds.time.dt.year >= 1948) & (ds.time.dt.year <= 2005)))
print(ds.dims)
print(ds.coords)
print()
ds = ds.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))
print(ds.dims)
print(ds.coords)
print()
ds.to_netcdf(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/CRU_temp.nc")
print("****************************************************************************")
print()

Frozen({'lon': 720, 'lat': 360, 'time': 840})
Coordinates:
  * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * time     (time) datetime64[ns] 1941-01-16 1941-02-15 ... 2010-12-16

Frozen({'lon': 720, 'lat': 360, 'time': 696})
Coordinates:
  * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * time     (time) datetime64[ns] 1948-01-16 1948-02-15 ... 2005-12-16

Frozen({'lon': 110, 'lat': 50, 'time': 696})
Coordinates:
  * lon      (lon) float64 -124.8 -124.2 -123.8 -123.2 ... -71.25 -70.75 -70.25
  * lat      (lat) float64 25.25 25.75 26.25 26.75 ... 48.25 48.75 49.25 49.75
  * time     (time) datetime64[ns] 1948-01-16 1948-02-15 ... 2005-12-16



# CRU .npy Regridded

In [8]:
ds = xr.open_dataset(r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/CRU_temp.nc")
dr = ds["tmp"]
ds_out = xr.Dataset(
    {
        "lat": (["lat"], lat_lst),
        "lon": (["lon"], lon_lst),
    }
)
regridder = xe.Regridder(ds, ds_out, "bilinear")
dr_out = regridder(dr)
dr_out = dr_out.fillna(0)
dr_np = dr_out.to_numpy()
np.save(
    r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/CRU_temp.npy", 
    dr_np
    )

# CMIP5 Data

In [9]:
for model in ['GFDL-CM3', 'MIROC-ESM', 'CanESM2']:
    files_list = glob.glob(rf"/home/uditbhatia/Documents/Sarth/CMIP5_tmp/tas_Amon_{model}_historical_r1i1p1_*.nc")
    print(f"{model}: {len(files_list)}")
    ds = xr.open_mfdataset(files_list,combine = 'by_coords')
    print(ds.dims)
    print(ds.coords)
    print()
    ds = ds.sel(time=((ds.time.dt.year >= 1948) & (ds.time.dt.year <= 2005)))
    print(ds.dims)
    print(ds.coords)
    print()
    ds = ds.sel(lat=slice(*lat_bnds), lon=slice(*[x%360 for x in lon_bnds]))
    print(ds.dims)
    print(ds.coords)
    print()
    ds.to_netcdf(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/{model}_mon_temp.nc")
    print("****************************************************************************")
    print()

GFDL-CM3: 13
Frozen({'time': 732, 'bnds': 2, 'lat': 90, 'lon': 144})
Coordinates:
  * time     (time) object 1945-01-16 12:00:00 ... 2005-12-16 12:00:00
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
    height   float64 2.0

Frozen({'time': 696, 'bnds': 2, 'lat': 90, 'lon': 144})
Coordinates:
  * time     (time) object 1948-01-16 12:00:00 ... 2005-12-16 12:00:00
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
    height   float64 2.0

Frozen({'time': 696, 'bnds': 2, 'lat': 13, 'lon': 22})
Coordinates:
  * time     (time) object 1948-01-16 12:00:00 ... 2005-12-16 12:00:00
  * lat      (lat) float64 25.0 27.0 29.0 31.0 33.0 ... 41.0 43.0 45.0 47.0 49.0
  * lon      (lon) float64 236.2 238.8 241.2 243.8 ... 281.2 283.8 286.2 288.8
    height   float64 2.0



/home/uditbhatia/anaconda3/envs/conda_esmf/lib/python3.7/site-packages/xarray/conventions.py:449: UserWarning: Variable 'time' has datetime type and a bounds variable but time.encoding does not have units specified. The units encodings for 'time' and 'time_bnds' will be determined independently and may not be equal, counter to CF-conventions. If this is a concern, specify a units encoding for 'time' before writing to a file.
  UserWarning,


****************************************************************************

MIROC-ESM: 1
Frozen({'time': 1872, 'bnds': 2, 'lat': 64, 'lon': 128})
Coordinates:
  * time     (time) datetime64[ns] 1850-01-16T12:00:00 ... 2005-12-16T12:00:00
  * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 79.53 82.31 85.1 87.86
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
    height   float64 ...

Frozen({'time': 696, 'bnds': 2, 'lat': 64, 'lon': 128})
Coordinates:
  * time     (time) datetime64[ns] 1948-01-16T12:00:00 ... 2005-12-16T12:00:00
  * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 79.53 82.31 85.1 87.86
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
    height   float64 ...

Frozen({'time': 696, 'bnds': 2, 'lat': 9, 'lon': 20})
Coordinates:
  * time     (time) datetime64[ns] 1948-01-16T12:00:00 ... 2005-12-16T12:00:00
  * lat      (lat) float64 26.51 29.3 32.09 34.88 37.67 40.46 43.25 46.04 48.84
  * lon      (lon)

In [11]:
for model in ['GFDL-CM3', 'MIROC-ESM', 'CanESM2']:
    ds = xr.open_dataset(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/{model}_mon_temp.nc")
    dr = ds["tas"]
    ds_out = xr.Dataset(
        {
            "lat": (["lat"], lat_lst),
            "lon": (["lon"], [x%360 for x in lon_lst]),
        }
    )
    regridder = xe.Regridder(ds, ds_out, "bilinear")
    dr_out = regridder(dr)
    dr_np = dr_out.to_numpy()
    np.save(
        rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/{model}_mon_temp.npy", 
        dr_np
        )

# Reanalysis Data

In [12]:
'''
Using that from CRU_01.ipynb
'''

'\nUsing that from CRU_01.ipynb\n'

# Stacking

In [13]:
CanESM2 = np.load(r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/CanESM2_mon_temp.npy")
MIROC_ESM = np.load(r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/MIROC-ESM_mon_temp.npy")
# HadGEM2_ES = np.load(r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/HadGEM2-ES_mon_temp.npy")
GFDL_CM3 = np.load(r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/GFDL-CM3_mon_temp.npy")

In [14]:
SRTM = np.load(r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/SRTM_US.npy")
SRTM = SRTM.reshape(1,51,111)
elevation = np.ones((696,51,111))
elevation = elevation*np.array(SRTM)

In [15]:
rhum = np.load(r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/rhum_mon.npy")
pres = np.load(r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/pres_mon.npy")
uwnd = np.load(r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/uwnd_mon.npy")
vwnd = np.load(r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/vwnd_mon.npy")
omega = np.load(r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/omega_mon.npy")

In [17]:
X = np.zeros((7,696,129,135))
X[0,:,39:90,12:123] = np.array(CanESM2)
X[1,:,39:90,12:123] = np.array(elevation)
X[2,:,39:90,12:123] = np.array(rhum)
X[3,:,39:90,12:123] = np.array(pres)
X[4,:,39:90,12:123] = np.array(uwnd)
X[5,:,39:90,12:123] = np.array(vwnd)
X[6,:,39:90,12:123] = np.array(omega)
np.save(
    r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/Stacked/CanESM2_mon_temp.npy", 
    X
    )

In [18]:
X = np.zeros((7,696,129,135))
X[0,:,39:90,12:123] = np.array(MIROC_ESM)
X[1,:,39:90,12:123] = np.array(elevation)
X[2,:,39:90,12:123] = np.array(rhum)
X[3,:,39:90,12:123] = np.array(pres)
X[4,:,39:90,12:123] = np.array(uwnd)
X[5,:,39:90,12:123] = np.array(vwnd)
X[6,:,39:90,12:123] = np.array(omega)
np.save(
    r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/Stacked/MIROC-ESM_mon_temp.npy", 
    X
    )

In [19]:
X = np.zeros((7,696,129,135))
X[0,:,39:90,12:123] = np.array(GFDL_CM3)
X[1,:,39:90,12:123] = np.array(elevation)
X[2,:,39:90,12:123] = np.array(rhum)
X[3,:,39:90,12:123] = np.array(pres)
X[4,:,39:90,12:123] = np.array(uwnd)
X[5,:,39:90,12:123] = np.array(vwnd)
X[6,:,39:90,12:123] = np.array(omega)
np.save(
    r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/Stacked/GFDL-CM3_mon_temp.npy", 
    X
    )

In [20]:
CRU_temp = np.load(r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/npy_Files/CRU_temp.npy")
CRU_temp.shape

(696, 51, 111)

In [21]:
Y = np.zeros((1,696,129,135))
Y[0,:,39:90,12:123] = np.array(CRU_temp)
np.save(
    r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/Stacked/Y_mon_temp.npy", 
    Y
    )